In [1]:
# ✅ Rule-Based Waste Classifier - Final Version for Jupyter
import json, difflib, logging
from pathlib import Path
from collections import Counter

# ========== CONFIG ==========
RULES_FILE = Path("waste_rules.json")
LOG_FILE = Path("classifier.log")
FUZZY_CUTOFF = 0.8

logging.basicConfig(filename=LOG_FILE, level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# ========== Default Rules & Synonyms ==========
DEFAULT_RULES = {
    "banana peel": ("Organic", "Compost"),
    "plastic bottle": ("Plastic", "Recycle"),
    "battery": ("Hazardous", "Dispose at collection point"),
    "glass jar": ("Glass", "Recycle"),
    "egg shell": ("Organic", "Compost"),
    "can": ("Metal", "Recycle"),
    "newspaper": ("Paper", "Recycle"),
    "food wrapper": ("Plastic", "Dispose")
}

SYNONYMS = {
    "bottle": "plastic bottle",
    "tin can": "can",
    "wrapper": "food wrapper",
    "bulb": "light bulb"
}

# ========== Helpers ==========
def load_rules():
    if RULES_FILE.exists():
        with open(RULES_FILE, "r") as f:
            return json.load(f)
    save_rules(DEFAULT_RULES)
    return DEFAULT_RULES.copy()

def save_rules(rules):
    with open(RULES_FILE, "w") as f:
        json.dump(rules, f, indent=2)

def normalise(text):
    return "".join(ch.lower() for ch in text if ch.isalnum() or ch.isspace()).strip()

# ========== Classifier ==========
class WasteClassifier:
    def __init__(self):
        self.rules = load_rules()
        self.synonyms = SYNONYMS

    def classify(self, raw_item):
        term = normalise(raw_item)
        term = self.synonyms.get(term, term)
        if term in self.rules:
            return *self.rules[term], term
        possible = difflib.get_close_matches(term, self.rules.keys(), n=1, cutoff=FUZZY_CUTOFF)
        if possible:
            match = possible[0]
            return *self.rules[match], match
        logging.info(f"Unknown item: {raw_item}")
        return None, None, term

    def add_rule(self, item, category, action):
        item = normalise(item)
        self.rules[item] = (category, action)
        save_rules(self.rules)
        print(f"✅ Rule added: {item} → {category}, {action}")

    def evaluate(self, test_data):
        y_true, y_pred = [], []
        for item, truth in test_data.items():
            pred, _, _ = self.classify(item)
            if truth is not None:
                y_true.append(truth)
                y_pred.append(pred or "Unknown")

        labels = sorted(set(y_true + y_pred))
        cm = Counter(zip(y_true, y_pred))
        print("📊 Evaluation Metrics:\n")
        print(f"{'Label':<12}{'Precision':>10}{'Recall':>10}{'F1':>8}{'Support':>10}")
        print("-" * 50)

        total_correct, total_samples = 0, 0

        for label in labels:
            TP = cm.get((label, label), 0)
            FP = sum(cm.get((other, label), 0) for other in labels if other != label)
            FN = sum(cm.get((label, other), 0) for other in labels if other != label)
            support = TP + FN
            precision = TP / (TP + FP) if (TP + FP) else 0
            recall = TP / support if support else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
            print(f"{label:<12}{precision:>10.2f}{recall:>10.2f}{f1:>8.2f}{support:>10}")
            total_correct += TP
            total_samples += support

        accuracy = total_correct / total_samples * 100 if total_samples else 0
        print("\n✅ Accuracy: {:.2f}% ({}/{})\n".format(accuracy, total_correct, total_samples))

# ========== Create Instance ==========
clf = WasteClassifier()

# ========== 🔹 Demo Inputs (For Screenshot) ==========
print("🔹 DEMO: Required Example Inputs\n")
demo_items = ["banana peel", "plastic bottle", "battery"]
for item in demo_items:
    cat, act, match = clf.classify(item)
    print(f"Input: {item:<20} → Output: {cat}, {act}")

# ========== 🔹 Custom Inputs ==========
print("\n🔹 CUSTOM INPUTS\n")
custom_items = ["glass jar", "egg shell", "tin can", "coated paper"]
for item in custom_items:
    cat, act, match = clf.classify(item)
    print(f"Input: {item:<20} → Output: {cat or 'Unknown'}, {act or 'Unknown'}")

# ========== 🔹 Evaluation ==========
print("\n🔹 EVALUATION REPORT")
test_set = {
    "banana peel": "Organic",
    "plastic bottle": "Plastic",
    "battery": "Hazardous",
    "glass jar": "Glass",
    "egg shell": "Organic",
    "can": "Metal",
    "unknown item": None
}
clf.evaluate(test_set)

# ========== 🔹 Add New Rule Example ==========
print("🔹 ADD RULE\n")
clf.add_rule("coated paper", "Paper", "Recycle")

# Try the new rule
cat, act, match = clf.classify("coated paper")
print(f"Test New Rule: coated paper → {cat}, {act}")


🔹 DEMO: Required Example Inputs

Input: banana peel          → Output: Organic, Compost
Input: plastic bottle       → Output: Plastic, Recycle
Input: battery              → Output: Hazardous, Dispose at collection point

🔹 CUSTOM INPUTS

Input: glass jar            → Output: Glass, Recycle
Input: egg shell            → Output: Organic, Compost
Input: tin can              → Output: Metal, Recycle
Input: coated paper         → Output: Unknown, Unknown

🔹 EVALUATION REPORT
📊 Evaluation Metrics:

Label        Precision    Recall      F1   Support
--------------------------------------------------
Glass             1.00      1.00    1.00         1
Hazardous         1.00      1.00    1.00         1
Metal             1.00      1.00    1.00         1
Organic           1.00      1.00    1.00         2
Plastic           1.00      1.00    1.00         1

✅ Accuracy: 100.00% (6/6)

🔹 ADD RULE

✅ Rule added: coated paper → Paper, Recycle
Test New Rule: coated paper → Paper, Recycle
